In [ ]:
# qualifiers to include: https://github.com/biolink/biolink-model/issues/1050

In [2]:
import pandas as pd
import json
import sys, os
import numpy as np

In [3]:
# nodes_filepath = os.path.join("../../data", "ehr_risk_nodes_data_2023_03_09.csv")
# edges_filepath = os.path.join("../../data", "ehr_risk_edges_data_2023_03_09.csv")
nodes_filepath = os.path.join("../../data", "ehr_risk_nodes_data_2022_06_01.csv")
edges_filepath = os.path.join("../../data", "ehr_risk_edges_data_2022_06_01.csv")
nodes_data = pd.read_csv(nodes_filepath, sep = ',')
edges_data = pd.read_csv(edges_filepath, sep = ',')

nodes_data

,id,name,category,xref,provided_by
0,HP:0025181,Abdominal aseptic abscess,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
1,HP:0002027,Abdominal pain,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
2,HP:0003115,Abnormal EKG,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
3,HP:0006919,"Abnormal aggressive, impulsive or violent beha...",biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
4,HP:0003119,Abnormal circulating lipid concentration,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
...,...,...,...,...,...
801,CHEBI:27300,vitamin d,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)
802,CHEBI:33234,vitamin e,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)
803,CHEBI:87732,warfarin,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)
804,CHEBI:36560,zinc oxide,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)


In [3]:
nodes_data.loc[nodes_data['name'].isin(['hypogonadism, male'])]

,id,name,category,xref,provided_by
511,MONDO:0009421,"hypogonadism, male",biolink:Disease,NaN,EHR Risk Provider (Multiomics)


# There appears to be duplicate CURIEs 
## This should be fixed at the enclave level

In [5]:
# there appears to be multiple CURIEs given for a node of the same name. Get just 1 CURIE for each name
nodes_data = nodes_data.drop_duplicates(subset='id', keep="first")
nodes_data

,id,name,category,xref,provided_by
0,HP:0025181,Abdominal aseptic abscess,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
1,HP:0002027,Abdominal pain,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
2,HP:0003115,Abnormal EKG,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
3,HP:0006919,"Abnormal aggressive, impulsive or violent beha...",biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
4,HP:0003119,Abnormal circulating lipid concentration,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
...,...,...,...,...,...
801,CHEBI:27300,vitamin d,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)
802,CHEBI:33234,vitamin e,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)
803,CHEBI:87732,warfarin,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)
804,CHEBI:36560,zinc oxide,biolink:ChemicalSubstance,NaN,EHR Risk Provider (Multiomics)


In [6]:
# biolink:ChemicalSubstance has been depecrated. Use biolink:ChemicalEntity instead (change all values ChemicalSubstance--->ChemialEntity)
nodes_data["category"].mask(nodes_data["category"] == "biolink:ChemicalSubstance", "biolink:ChemicalEntity" , inplace=True )
nodes_data


,id,name,category,xref,provided_by
0,HP:0025181,Abdominal aseptic abscess,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
1,HP:0002027,Abdominal pain,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
2,HP:0003115,Abnormal EKG,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
3,HP:0006919,"Abnormal aggressive, impulsive or violent beha...",biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
4,HP:0003119,Abnormal circulating lipid concentration,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
...,...,...,...,...,...
801,CHEBI:27300,vitamin d,biolink:ChemicalEntity,NaN,EHR Risk Provider (Multiomics)
802,CHEBI:33234,vitamin e,biolink:ChemicalEntity,NaN,EHR Risk Provider (Multiomics)
803,CHEBI:87732,warfarin,biolink:ChemicalEntity,NaN,EHR Risk Provider (Multiomics)
804,CHEBI:36560,zinc oxide,biolink:ChemicalEntity,NaN,EHR Risk Provider (Multiomics)


In [7]:
# Check if the xref column is all NaN or empty
# get count of null values 
nodes_data['xref'].isna().sum()

# since xref is empty, we won't use it

804

In [8]:
edges_data

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,p_value,feature_importance,feature_coefficient,log_positive_patient_count,log_negative_patient_count
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,NaN,8.796399,5,7
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,NaN,8.585212,5,7
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,NaN,4.558177,5,7
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,NaN,4.357522,5,7
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,NaN,3.926064,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,MONDO:0013600,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.245984,NaN,-0.790418,5,7
237100,MONDO:0016264,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.984249,NaN,-0.796085,5,7
237101,MONDO:0004565,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.820564,NaN,-0.803973,5,7
237102,CHEBI:28864,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.472603,NaN,-0.822575,5,7


In [9]:
# sanity check on predicates
edges_data['predicate'].unique()

array(['biolink:associated_with_increased_likelihood_of',
       'biolink:associated_with_decreased_likelihood_of'], dtype=object)

In [10]:
edges_data["num_patients_with_condition"] = 10**(edges_data['log_positive_patient_count']) # convert log pos patient count to an actual # 
edges_data["num_patients_without_condition"] = 10**(edges_data['log_negative_patient_count']) # convert log neg patient count to an actual # 
edges_data["num_patients_with_condition"] = np.random.poisson(edges_data['num_patients_with_condition'])  # add poisson noise injection
edges_data["num_patients_without_condition"] = np.random.poisson(edges_data['num_patients_without_condition']) # add poisson noise injection
edges_data = edges_data.drop(['log_positive_patient_count', 'log_negative_patient_count'], axis=1)
edges_data

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,p_value,feature_importance,feature_coefficient,num_patients_with_condition,num_patients_without_condition
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,NaN,8.796399,99669,9999902
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,NaN,8.585212,99938,10000835
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,NaN,4.558177,99770,10000939
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,NaN,4.357522,99910,9998659
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,NaN,3.926064,100242,9998750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,MONDO:0013600,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.245984,NaN,-0.790418,99922,9995233
237100,MONDO:0016264,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.984249,NaN,-0.796085,99822,10002245
237101,MONDO:0004565,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.820564,NaN,-0.803973,99937,9999068
237102,CHEBI:28864,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.472603,NaN,-0.822575,99588,10000517


In [11]:
# Convert columns A and B to frozenset
# df['AB'] = df[['A', 'B']].apply(frozenset, axis=1)
edges_data['nodes_frozenset'] = edges_data[['subject', 'object']].apply(frozenset, axis=1)
edges_data

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,p_value,feature_importance,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,NaN,8.796399,99669,9999902,"(HP:0000360, HP:0008629)"
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,NaN,8.585212,99938,10000835,"(HP:0000360, MONDO:0010643)"
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,NaN,4.558177,99770,10000939,"(UNII:25ADE2236L, HP:0000360)"
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,NaN,4.357522,99910,9998659,"(HP:0000360, UNII:K16AIQ8CTM)"
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,NaN,3.926064,100242,9998750,"(MONDO:0007972, HP:0000360)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,MONDO:0013600,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.245984,NaN,-0.790418,99922,9995233,"(MONDO:0013600, HP:0033106)"
237100,MONDO:0016264,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.984249,NaN,-0.796085,99822,10002245,"(MONDO:0016264, HP:0033106)"
237101,MONDO:0004565,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.820564,NaN,-0.803973,99937,9999068,"(HP:0033106, MONDO:0004565)"
237102,CHEBI:28864,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.472603,NaN,-0.822575,99588,10000517,"(CHEBI:28864, HP:0033106)"


In [12]:
# Find duplicate rows based on frozenset of columns A and B
duplicate_rows = edges_data.duplicated(['nodes_frozenset', 'auc_roc', 'p_value', 'feature_coefficient'])

# Filter the DataFrame to remove duplicate rows
test = edges_data[~duplicate_rows]

test = test.drop(columns='nodes_frozenset')

test

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,p_value,feature_importance,feature_coefficient,num_patients_with_condition,num_patients_without_condition
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,NaN,8.796399,99669,9999902
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,NaN,8.585212,99938,10000835
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,NaN,4.558177,99770,10000939
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,NaN,4.357522,99910,9998659
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,NaN,3.926064,100242,9998750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,MONDO:0013600,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.245984,NaN,-0.790418,99922,9995233
237100,MONDO:0016264,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.984249,NaN,-0.796085,99822,10002245
237101,MONDO:0004565,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.820564,NaN,-0.803973,99937,9999068
237102,CHEBI:28864,biolink:associated_with_decreased_likelihood_of,HP:0033106,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.956860,0.472603,NaN,-0.822575,99588,10000517


In [40]:
# # create confidence interval column by concatenating 'lower_confidence_bound'and 'upper_confidence_bound', then dropping those columns
# edges_data['log_odds_ratio_95_confidence_interval'] = edges_data.apply(lambda row: [row['lower_confidence_bound'], row['upper_confidence_bound']], axis=1)
# edges_data = edges_data.drop(['lower_confidence_bound', 'upper_confidence_bound'], axis=1)
# edges_data

In [13]:
kg = pd.merge(edges_data, nodes_data[['id', 'name', 'category']], left_on='subject', right_on = 'id', how="inner")
kg

,subject,predicate,object,relation,provided_by,provided_date,category_x,classifier,auc_roc,p_value,feature_importance,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,id,name,category_y
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,NaN,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
1,HP:0008629,biolink:associated_with_decreased_likelihood_of,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.876100,0.985604,NaN,-0.530062,99841,9999688,"(HP:0008629, HP:0000739)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
2,HP:0008629,biolink:associated_with_decreased_likelihood_of,HP:0000787,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.841102,0.981654,NaN,-1.180669,100354,10002196,"(HP:0008629, HP:0000787)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
3,HP:0008629,biolink:associated_with_decreased_likelihood_of,HP:0000790,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.910838,0.960982,NaN,-1.200019,99835,10001455,"(HP:0008629, HP:0000790)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
4,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000870,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.884580,0.988524,NaN,1.432611,968,9996545,"(HP:0008629, HP:0000870)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0032312,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.921417,0.927675,NaN,-0.171607,99721,10001233,"(HP:0000360, HP:0032312)",HP:0000360,Tinnitus,biolink:PhenotypicFeature
237100,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0032372,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.990023,0.962012,NaN,-2.817568,1041,9998402,"(HP:0000360, HP:0032372)",HP:0000360,Tinnitus,biolink:PhenotypicFeature
237101,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0032473,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.963206,0.989129,NaN,-5.438955,8,10000998,"(HP:0000360, HP:0032473)",HP:0000360,Tinnitus,biolink:PhenotypicFeature
237102,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0033077,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.987230,0.541701,NaN,-1.017309,100548,10003263,"(HP:0000360, HP:0033077)",HP:0000360,Tinnitus,biolink:PhenotypicFeature


In [14]:
kg.rename(columns = {'category_x':'predicate_category',
                     'category_y': 'subject_category',
                     'id': 'subject_id',
                     'name': 'subject_name'}, inplace = True)
kg

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,feature_importance,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,NaN,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
1,HP:0008629,biolink:associated_with_decreased_likelihood_of,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.876100,0.985604,NaN,-0.530062,99841,9999688,"(HP:0008629, HP:0000739)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
2,HP:0008629,biolink:associated_with_decreased_likelihood_of,HP:0000787,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.841102,0.981654,NaN,-1.180669,100354,10002196,"(HP:0008629, HP:0000787)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
3,HP:0008629,biolink:associated_with_decreased_likelihood_of,HP:0000790,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.910838,0.960982,NaN,-1.200019,99835,10001455,"(HP:0008629, HP:0000790)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
4,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000870,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.884580,0.988524,NaN,1.432611,968,9996545,"(HP:0008629, HP:0000870)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0032312,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.921417,0.927675,NaN,-0.171607,99721,10001233,"(HP:0000360, HP:0032312)",HP:0000360,Tinnitus,biolink:PhenotypicFeature
237100,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0032372,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.990023,0.962012,NaN,-2.817568,1041,9998402,"(HP:0000360, HP:0032372)",HP:0000360,Tinnitus,biolink:PhenotypicFeature
237101,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0032473,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.963206,0.989129,NaN,-5.438955,8,10000998,"(HP:0000360, HP:0032473)",HP:0000360,Tinnitus,biolink:PhenotypicFeature
237102,HP:0000360,biolink:associated_with_decreased_likelihood_of,HP:0033077,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.987230,0.541701,NaN,-1.017309,100548,10003263,"(HP:0000360, HP:0033077)",HP:0000360,Tinnitus,biolink:PhenotypicFeature


In [15]:
# merging object info from nodes df
kg = pd.merge(kg, nodes_data[['id', 'name', 'category']], left_on='object', right_on = 'id', how="inner")
kg

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,...,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category,id,name,category
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,...,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,HP:0000360,Tinnitus,biolink:PhenotypicFeature
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,...,8.585212,99938,10000835,"(HP:0000360, MONDO:0010643)",MONDO:0010643,acute leukemia (disease),biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,...,4.558177,99770,10000939,"(UNII:25ADE2236L, HP:0000360)",UNII:25ADE2236L,thrombin,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,...,4.357522,99910,9998659,"(HP:0000360, UNII:K16AIQ8CTM)",UNII:K16AIQ8CTM,pertuzumab,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,...,3.926064,100242,9998750,"(MONDO:0007972, HP:0000360)",MONDO:0007972,Meniere disease,biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,CHEBI:114785,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998503,...,-5.087542,1007,10007299,"(HP:0008629, CHEBI:114785)",CHEBI:114785,erlotinib,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237100,UNII:52CMI0WC3Y,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.999719,...,-5.066445,1016,9996273,"(UNII:52CMI0WC3Y, HP:0008629)",UNII:52CMI0WC3Y,atezolizumab,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237101,CHEBI:135738,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998357,...,-5.079640,978,9998030,"(HP:0008629, CHEBI:135738)",CHEBI:135738,clevidipine,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237102,MONDO:0004967,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.997631,...,-4.683547,1006,10001385,"(HP:0008629, MONDO:0004967)",MONDO:0004967,acute lymphoblastic leukemia (disease),biolink:Disease,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature


In [16]:
kg.rename(columns = {'id':'object_id',
                     'category': 'object_category',
                     'name': 'object_name'}, inplace = True)
kg

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,...,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category,object_id,object_name,object_category
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,...,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,HP:0000360,Tinnitus,biolink:PhenotypicFeature
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,...,8.585212,99938,10000835,"(HP:0000360, MONDO:0010643)",MONDO:0010643,acute leukemia (disease),biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,...,4.558177,99770,10000939,"(UNII:25ADE2236L, HP:0000360)",UNII:25ADE2236L,thrombin,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,...,4.357522,99910,9998659,"(HP:0000360, UNII:K16AIQ8CTM)",UNII:K16AIQ8CTM,pertuzumab,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,...,3.926064,100242,9998750,"(MONDO:0007972, HP:0000360)",MONDO:0007972,Meniere disease,biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,CHEBI:114785,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998503,...,-5.087542,1007,10007299,"(HP:0008629, CHEBI:114785)",CHEBI:114785,erlotinib,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237100,UNII:52CMI0WC3Y,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.999719,...,-5.066445,1016,9996273,"(UNII:52CMI0WC3Y, HP:0008629)",UNII:52CMI0WC3Y,atezolizumab,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237101,CHEBI:135738,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998357,...,-5.079640,978,9998030,"(HP:0008629, CHEBI:135738)",CHEBI:135738,clevidipine,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237102,MONDO:0004967,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.997631,...,-4.683547,1006,10001385,"(HP:0008629, MONDO:0004967)",MONDO:0004967,acute lymphoblastic leukemia (disease),biolink:Disease,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature


In [17]:
# ENSURE THERE ARE UNIQUE RECORDS/ROWS
kg = kg.drop_duplicates(['subject', 'object', 'auc_roc', 'p_value', 'feature_coefficient'], keep='first')
kg

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,...,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category,object_id,object_name,object_category
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,...,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,HP:0000360,Tinnitus,biolink:PhenotypicFeature
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,...,8.585212,99938,10000835,"(HP:0000360, MONDO:0010643)",MONDO:0010643,acute leukemia (disease),biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,...,4.558177,99770,10000939,"(UNII:25ADE2236L, HP:0000360)",UNII:25ADE2236L,thrombin,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,...,4.357522,99910,9998659,"(HP:0000360, UNII:K16AIQ8CTM)",UNII:K16AIQ8CTM,pertuzumab,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,...,3.926064,100242,9998750,"(MONDO:0007972, HP:0000360)",MONDO:0007972,Meniere disease,biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,CHEBI:114785,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998503,...,-5.087542,1007,10007299,"(HP:0008629, CHEBI:114785)",CHEBI:114785,erlotinib,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237100,UNII:52CMI0WC3Y,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.999719,...,-5.066445,1016,9996273,"(UNII:52CMI0WC3Y, HP:0008629)",UNII:52CMI0WC3Y,atezolizumab,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237101,CHEBI:135738,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998357,...,-5.079640,978,9998030,"(HP:0008629, CHEBI:135738)",CHEBI:135738,clevidipine,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237102,MONDO:0004967,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.997631,...,-4.683547,1006,10001385,"(HP:0008629, MONDO:0004967)",MONDO:0004967,acute lymphoblastic leukemia (disease),biolink:Disease,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature


In [18]:
kg_NONE_subjects = kg[kg["subject"].str.contains("NONE")==True]
print(len(kg_NONE_subjects))
kg_NONE_objects = kg[kg["object"].str.contains("NONE")==True]
print(len(kg_NONE_objects))

292
0


In [19]:
# for some reason, some subject or object ids are empty (they actually literally contain the string "NONE")
kg = kg.dropna(axis=0, subset=['subject'])
kg = kg.dropna(axis=0, subset=['object'])
kg = kg[~kg["subject"].str.contains("NONE")==True]
kg = kg[~kg["object"].str.contains("NONE")==True]
kg

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,...,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category,object_id,object_name,object_category
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,...,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,HP:0000360,Tinnitus,biolink:PhenotypicFeature
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,...,8.585212,99938,10000835,"(HP:0000360, MONDO:0010643)",MONDO:0010643,acute leukemia (disease),biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,...,4.558177,99770,10000939,"(UNII:25ADE2236L, HP:0000360)",UNII:25ADE2236L,thrombin,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,...,4.357522,99910,9998659,"(HP:0000360, UNII:K16AIQ8CTM)",UNII:K16AIQ8CTM,pertuzumab,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,...,3.926064,100242,9998750,"(MONDO:0007972, HP:0000360)",MONDO:0007972,Meniere disease,biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,CHEBI:114785,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998503,...,-5.087542,1007,10007299,"(HP:0008629, CHEBI:114785)",CHEBI:114785,erlotinib,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237100,UNII:52CMI0WC3Y,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.999719,...,-5.066445,1016,9996273,"(UNII:52CMI0WC3Y, HP:0008629)",UNII:52CMI0WC3Y,atezolizumab,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237101,CHEBI:135738,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998357,...,-5.079640,978,9998030,"(HP:0008629, CHEBI:135738)",CHEBI:135738,clevidipine,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237102,MONDO:0004967,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.997631,...,-4.683547,1006,10001385,"(HP:0008629, MONDO:0004967)",MONDO:0004967,acute lymphoblastic leukemia (disease),biolink:Disease,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature


In [20]:
# for some reason, some subject or object ids are empty (they actually literally contain the string "NONE")
# get rid of these rows
kg = kg[~kg["subject"].str.contains("NONE")==True]  # subject and object are all CURIEs, not names
kg = kg[~kg["object"].str.contains("NONE")==True]
kg = kg[~kg["subject"].str.contains("none")==True]
kg = kg[~kg["object"].str.contains("none")==True]
kg = kg[~kg["subject"].str.contains("None")==True]
kg = kg[~kg["object"].str.contains("None")==True]
kg

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,...,feature_coefficient,num_patients_with_condition,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category,object_id,object_name,object_category
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,...,8.796399,99669,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,HP:0000360,Tinnitus,biolink:PhenotypicFeature
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,...,8.585212,99938,10000835,"(HP:0000360, MONDO:0010643)",MONDO:0010643,acute leukemia (disease),biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,...,4.558177,99770,10000939,"(UNII:25ADE2236L, HP:0000360)",UNII:25ADE2236L,thrombin,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,...,4.357522,99910,9998659,"(HP:0000360, UNII:K16AIQ8CTM)",UNII:K16AIQ8CTM,pertuzumab,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,...,3.926064,100242,9998750,"(MONDO:0007972, HP:0000360)",MONDO:0007972,Meniere disease,biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,CHEBI:114785,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998503,...,-5.087542,1007,10007299,"(HP:0008629, CHEBI:114785)",CHEBI:114785,erlotinib,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237100,UNII:52CMI0WC3Y,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.999719,...,-5.066445,1016,9996273,"(UNII:52CMI0WC3Y, HP:0008629)",UNII:52CMI0WC3Y,atezolizumab,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237101,CHEBI:135738,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998357,...,-5.079640,978,9998030,"(HP:0008629, CHEBI:135738)",CHEBI:135738,clevidipine,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature
237102,MONDO:0004967,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.997631,...,-4.683547,1006,10001385,"(HP:0008629, MONDO:0004967)",MONDO:0004967,acute lymphoblastic leukemia (disease),biolink:Disease,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature


In [ ]:
# OPTIONAL THRESHOLD
# kg_pval_subsetted = kg[kg["p_value"] < 0.2]
# kg_pval_subsetted

In [334]:
# get a list of meta-triples for this KP: make it look like what BTE wants as closely as possible (see yaml)
# X_BTE KNOWN OPERATIONS TO ADD
#     unique combos of subject-prefix / subject-category / predicate / object-prefix / object-category 
#     (and qualifier-set, if applicable)

# Use the grouped["joined_to_match_xbte"] column made below to create the tags and x-bte-kgs-operations portions in ehr_risk_kg.yaml
    
meta_triples = kg.copy()

meta_triples["subject_prefix"] = meta_triples["subject"].str.split(":").str[0]
meta_triples["object_prefix"] = meta_triples["object"].str.split(":").str[0]

meta_triples["subject_category_postfix"] = meta_triples["subject_category"].str.split(":").str[1]
meta_triples["object_category_postfix"] = meta_triples["object_category"].str.split(":").str[1]

meta_triples['increased_or_decreased'] = None
meta_triples.loc[meta_triples['predicate'].str.contains('increased', case=False, na=False), 'increased_or_decreased'] = 'increased'
meta_triples.loc[meta_triples['predicate'].str.contains('decreased', case=False, na=False), 'increased_or_decreased'] = 'decreased'

# unique combos of subject-prefix / subject-category / predicate / object-prefix / object-category

# Define the columns you want to combine
columns_to_combine = ['subject_category_postfix', 'subject_prefix', 'increased_or_decreased', 'object_category_postfix', 'object_prefix']

# Group by the combination of columns and count the occurrences
grouped = meta_triples.groupby(columns_to_combine).size().reset_index(name='Count')
grouped['object_category_postfix_abbv'] = grouped['object_category_postfix'].str.replace('PhenotypicFeature', 'Pheno')
grouped['subject_category_postfix_abbv'] = grouped['subject_category_postfix'].str.replace('PhenotypicFeature', 'Pheno')

grouped['object_category_postfix_abbv'] = grouped['object_category_postfix_abbv'].str.replace('ChemicalEntity', 'Chem')
grouped['subject_category_postfix_abbv'] = grouped['subject_category_postfix_abbv'].str.replace('ChemicalEntity', 'Chem')

grouped['joined_to_match_xbte'] = grouped['subject_category_postfix_abbv'].astype(str) + grouped['subject_prefix'].astype(str) + '_' + grouped['increased_or_decreased'].astype(str) + '_' + grouped['object_category_postfix_abbv'].astype(str) + grouped['object_prefix'].astype(str)

# grouped.to_csv('yaml_all_metatriples.tsv', sep="\t")
# # Print the DataFrame with unique combinations and their counts

with pd.option_context("display.max_rows", 30000):
    display(grouped)
    

,subject_category_postfix,subject_prefix,increased_or_decreased,object_category_postfix,object_prefix,Count,object_category_postfix_abbv,subject_category_postfix_abbv,joined_to_match_xbte
0,ChemicalEntity,CHEBI,decreased,Disease,MONDO,34917,Disease,Chem,ChemCHEBI_decreased_DiseaseMONDO
1,ChemicalEntity,CHEBI,decreased,Disease,NCIT,212,Disease,Chem,ChemCHEBI_decreased_DiseaseNCIT
2,ChemicalEntity,CHEBI,decreased,Disease,SNOMEDCT,2917,Disease,Chem,ChemCHEBI_decreased_DiseaseSNOMEDCT
3,ChemicalEntity,CHEBI,decreased,PhenotypicFeature,HP,24505,Pheno,Chem,ChemCHEBI_decreased_PhenoHP
4,ChemicalEntity,CHEBI,decreased,PhenotypicFeature,NCIT,2254,Pheno,Chem,ChemCHEBI_decreased_PhenoNCIT
5,ChemicalEntity,CHEBI,decreased,PhenotypicFeature,SNOMEDCT,244,Pheno,Chem,ChemCHEBI_decreased_PhenoSNOMEDCT
6,ChemicalEntity,CHEBI,decreased,Procedure,NCIT,222,Procedure,Chem,ChemCHEBI_decreased_ProcedureNCIT
7,ChemicalEntity,CHEBI,increased,Disease,MONDO,18248,Disease,Chem,ChemCHEBI_increased_DiseaseMONDO
8,ChemicalEntity,CHEBI,increased,Disease,NCIT,131,Disease,Chem,ChemCHEBI_increased_DiseaseNCIT
9,ChemicalEntity,CHEBI,increased,Disease,SNOMEDCT,1542,Disease,Chem,ChemCHEBI_increased_DiseaseSNOMEDCT


In [335]:
# need this (dataframe called kg_yaml) to lookup examples for testExamples (qInput and oneOutput) portion of yaml
# additionally need this generate portion of x-bte-response-mapping

kg_yaml = kg.copy()

kg_yaml["subject_prefix"] = kg_yaml["subject"].str.split(":").str[0]
kg_yaml["object_prefix"] = kg_yaml["object"].str.split(":").str[0]

kg_yaml

,subject,predicate,object,relation,provided_by,provided_date,predicate_category,classifier,auc_roc,p_value,...,num_patients_without_condition,nodes_frozenset,subject_id,subject_name,subject_category,object_id,object_name,object_category,subject_prefix,object_prefix
0,HP:0008629,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.000000,...,9999902,"(HP:0000360, HP:0008629)",HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,HP:0000360,Tinnitus,biolink:PhenotypicFeature,HP,HP
1,MONDO:0010643,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.999872,...,10000835,"(HP:0000360, MONDO:0010643)",MONDO:0010643,acute leukemia (disease),biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature,MONDO,HP
2,UNII:25ADE2236L,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.936767,...,10000939,"(UNII:25ADE2236L, HP:0000360)",UNII:25ADE2236L,thrombin,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature,UNII,HP
3,UNII:K16AIQ8CTM,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.998563,...,9998659,"(HP:0000360, UNII:K16AIQ8CTM)",UNII:K16AIQ8CTM,pertuzumab,biolink:ChemicalEntity,HP:0000360,Tinnitus,biolink:PhenotypicFeature,UNII,HP
4,MONDO:0007972,biolink:associated_with_increased_likelihood_of,HP:0000360,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.840132,0.093959,...,9998750,"(MONDO:0007972, HP:0000360)",MONDO:0007972,Meniere disease,biolink:Disease,HP:0000360,Tinnitus,biolink:PhenotypicFeature,MONDO,HP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237099,CHEBI:114785,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998503,...,10007299,"(HP:0008629, CHEBI:114785)",CHEBI:114785,erlotinib,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,CHEBI,HP
237100,UNII:52CMI0WC3Y,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.999719,...,9996273,"(UNII:52CMI0WC3Y, HP:0008629)",UNII:52CMI0WC3Y,atezolizumab,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,UNII,HP
237101,CHEBI:135738,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.998357,...,9998030,"(HP:0008629, CHEBI:135738)",CHEBI:135738,clevidipine,biolink:ChemicalEntity,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,CHEBI,HP
237102,MONDO:0004967,biolink:associated_with_decreased_likelihood_of,HP:0008629,RO:0003308,EHR Risk Provider (Multiomics),2022-05-18,biolink:Association,Logistic Regression,0.959791,0.997631,...,10001385,"(HP:0008629, MONDO:0004967)",MONDO:0004967,acute lymphoblastic leukemia (disease),biolink:Disease,HP:0008629,Pulsatile tinnitus,biolink:PhenotypicFeature,MONDO,HP


In [339]:
# the following code writes the tags and x-bte-kgs-operations of ehr_risk_kg.yaml to an output file titled 'x-bte-kgs-operations-response-mapping.txt'
# copy paste the output to the relevant section in the yaml
with open('x-bte-kgs-operations-response-mapping.txt', 'w+') as file:
    file.write("      tags:" + "\n")
    file.write("      - query:" + "\n")
    file.write(f"      ## {len(grouped)} operations based on TSV of KG" + "\n")
    for index, row in grouped.iterrows():
        file.write(f"      ## - {row['joined_to_match_xbte']}" + "\n")
    file.write("      x-bte-kgs-operations" + "\n")
    for index, row in grouped.iterrows(): 
        file.write(f"      - $ref: '#/components/x-bte-kgs-operations/{row['joined_to_match_xbte']}'" + "\n")
        file.write(f"      - $ref: '#/components/x-bte-kgs-operations/{row['joined_to_match_xbte']}-rev'" + "\n")
        

In [340]:
# the following code writes the x-bte-kgs-operations of ehr_risk_kg.yaml to an output file titled 'x-bte-kgs-operations-response-mapping.txt'
# copy paste the output to the relevant section in the yaml

with open('x-bte-kgs-operations-response-mapping.txt', 'a') as file:
    file.write("  " + "x-bte-kgs-operations:" + "\n")
    for index, row in grouped.iterrows():
        xbte_header = f'{row["joined_to_match_xbte"]}:'
        file.write("    " + xbte_header + "\n")
        
        base_url = "https://pending.biothings.io/multiomics_ehr_risk_kp/query?q="
        url = base_url + f'''subject.type:"biolink:{row['subject_category_postfix']}"%20AND%20_exists_:subject."biolink:{row['subject_prefix']}"%20AND%20association.predicate:associated_with_decreased_likelihood_of%20AND%20object.type:"biolink:{row['object_category_postfix']}"%20AND%20_exists_:object.{row['object_prefix']}'''
        commented_url = "## " + url
        file.write("    " + commented_url + "\n")
        
        file.write("    ## " + str(row["Count"]) + " records" + "\n")
        
        file.write("      -supportBatch: true" + "\n")
        file.write("       useTemplating: true" + "\n")
        file.write("       inputs:" + "\n")
        
        file.write(f"          - id: {row['subject_prefix']}" + "\n")
        file.write(f"          - semantic: {row['subject_category_postfix']}" + "\n")
        
        file.write("       requestBody:" + "\n")
        file.write("         body: >-" + "\n")
        
        file.write("           {" + "\n")
        q_string = f""" "q": [ {{ queryInputs | rmPrefix() | wrap ( '["' , '","biolink:{row['subject_category_postfix']}", "associated_with_{row['increased_or_decreased']}_likelihood_of", "biolink:{row['object_category_postfix']}"]') }} ]"""
        file.write('              ' + q_string + "\n") 
        scopes_string = f""" "scopes": ["subject.{row['subject_prefix']}", "subject.type", "association.predicate", "object.type"]"""
        file.write('              ' + scopes_string + "\n") 
        file.write("           }" + "\n")
        
        file.write("       parameters:" + "\n")
        file.write(f"         fields: object.{row['object_prefix']}, association.edge_attributes, subject.name, object.name, source.edge_sources" + "\n")
        file.write(f"         size: 1000" + "\n")
        
        file.write("       outputs:" + "\n")
        file.write(f"         -id: {row['object_prefix']}" + "\n")
        file.write(f"          semantic: {row['object_category_postfix']}" + "\n")
        
        file.write("       predicate: associated_with" + "\n")
        file.write("       qualifiers:" + "\n")
        file.write(f"         object_direction_qualifier: {row['increased_or_decreased']}" + "\n")
        file.write(f"         object_aspect_qualifier: likelihood" + "\n")
        
        file.write("       response_mapping:" + "\n")
        file.write(f'         "ref": "#/components/x-bte-response-mapping/object-{row["object_prefix"]}"' + "\n")
        
        file.write("       # testExamples:" + "\n")
        sub_prefix = row["subject_prefix"]
        obj_prefix = row["object_prefix"]
        matched_rows = kg_yaml.loc[(kg_yaml['subject_prefix'] == sub_prefix) & (kg_yaml['object_prefix'] == obj_prefix)]
        select_row = matched_rows.sample()
        qInput = select_row['subject'].values[0]
        qName = select_row['subject_name'].values[0]
        oneOutput = select_row['object'].values[0]
        oneOutputName = select_row['object_name'].values[0]
        file.write(f'       #   - qInput: "{qInput}"      ## {qName} ' + "\n")
        file.write(f'       #     oneOutput: "{oneOutput}"      ## {oneOutputName} ' + "\n")
        
        #### DO REVERSE #### 
        
        xbte_header_rev = f'{row["joined_to_match_xbte"]}_rev:'
        file.write("    " + xbte_header_rev + "\n")

        file.write("      -supportBatch: true" + "\n")
        file.write("       useTemplating: true" + "\n")
        file.write("       inputs:" + "\n")
        
        file.write(f"          - id: {row['object_prefix']}" + "\n")
        file.write(f"          - semantic: {row['object_category_postfix']}" + "\n")
        
        file.write("       requestBody:" + "\n")
        file.write("         body: >-" + "\n")
        
        file.write("           {" + "\n")
        q_string = f""" "q": [ {{ queryInputs | rmPrefix() | wrap ( '["' , '","biolink:{row['subject_category_postfix']}", "associated_with_{row['increased_or_decreased']}_likelihood_of", "biolink:{row['object_category_postfix']}"]') }} ]"""
        file.write('              ' + q_string + "\n") 
        scopes_string = f""" "scopes": ["object.{row['object_prefix']}", "subject.type", "association.predicate", "object.type"]"""
        file.write('              ' + scopes_string + "\n") 
        file.write("           }" + "\n")
        
        file.write("       parameters:" + "\n")
        file.write(f"         fields: subject.{row['subject_prefix']}, association.edge_attributes, subject.name, object.name, source.edge_sources" + "\n")
        file.write(f"         size: 1000" + "\n")
        
        file.write("       outputs:" + "\n")
        file.write(f"         -id: {row['subject_prefix']}" + "\n")
        file.write(f"          semantic: {row['subject_category_postfix']}" + "\n")
        
        file.write("       predicate: associated_with" + "\n")
        file.write("       qualifiers:" + "\n")
        file.write(f"         object_direction_qualifier: {row['increased_or_decreased']}" + "\n")
        file.write(f"         object_aspect_qualifier: likelihood" + "\n")
        
        file.write("       response_mapping:" + "\n")
        file.write(f'         "ref": "#/components/x-bte-response-mapping/object-{row["subject_prefix"]}"' + "\n")
        
        file.write("       # testExamples:" + "\n")
        sub_prefix = row["subject_prefix"]
        obj_prefix = row["object_prefix"]
        matched_rows = kg_yaml.loc[(kg_yaml['subject_prefix'] == sub_prefix) & (kg_yaml['object_prefix'] == obj_prefix)]
        select_row = matched_rows.sample()
        qInput = select_row['object'].values[0]
        qName = select_row['object_name'].values[0]
        oneOutput = select_row['subject'].values[0]
        oneOutputName = select_row['subject_name'].values[0]
        file.write(f'       #   - qInput: "{qInput}"      ## {qName} ' + "\n")
        file.write(f'       #     oneOutput: "{oneOutput}"      ## {oneOutputName} ' + "\n")
        
        

    

In [ ]:
Output example for x-bte-kgs-operations

    ChemCHEBI_decreased_PhenoNCIT:
    ## https://pending.biothings.io/multiomics_ehr_risk_kp/query?q=subject.type:ChemicalEntity%20AND%20_exists_:subject.CHEBI%20AND%20association.predicate:associated_with_decreased_likelihood_of%20AND%20object.type:PhenotypicFeature%20AND%20_exists_:object.NCIT
    ## 191 records
      - supportBatch: true
        useTemplating: true  
        inputs:
          - id: CHEBI
            semantic: SmallMolecule
        requestBodyType: object
        requestBody:
          body: >-
            {
              "q": [ {{ queryInputs | rmPrefix() | wrap ( '["' , '","biolink:ChemicalEntity", "associated_with_decreased_likelihood_of", "biolink:PhenotypicFeature"]') }} ],
              "scopes": ["subject.CHEBI", "subject.type", "association.predicate", "object.type"]
            }
        parameters:
          fields: >-
            object.NCIT,
            association.edge_attributes,
            subject.name,object.name
          size: 1000
        outputs:
          - id: NCIT
            semantic: PhenotypicFeature
        predicate: associated_with
        qualifiers:
          object_direction_qualifier: decreased
          object_aspect_qualifier: likelihood
        response_mapping:
          "$ref": "#/components/x-bte-response-mapping/object-NCIT"
        # testExamples:
        #   - qInput: "CHEBI:135866"        ## clindamycin
        #     oneOutput: "NCIT:C171647"     ## SARS Coronavirus 2 Positive
    ChemCHEBI_decreased_PhenoNCIT-rev:
      - supportBatch: true
        useTemplating: true  
        inputs:
          - id: NCIT
            semantic: PhenotypicFeature
        requestBodyType: object
        requestBody:
          body: >-
            {
              "q": [ {{ queryInputs | rmPrefix() | wrap( '["' , '","biolink:ChemicalEntity", "associated_with_decreased_likelihood_of", "biolink:PhenotypicFeature"]') }} ],
              "scopes": ["object.NCIT", "subject.type", "association.predicate", "object.type"]
            }
        parameters:
          fields: >-
            subject.CHEBI,
            association.edge_attributes,
            subject.name,object.name
          size: 1000
        outputs:
          - id: CHEBI
            semantic: SmallMolecule
        predicate: associated_with
        qualifiers:
          subject_direction_qualifier: decreased
          subject_aspect_qualifier: likelihood
        response_mapping:
          "$ref": "#/components/x-bte-response-mapping/subject-CHEBI"
        # testExamples:
        #   - qInput: "NCIT:C171647"          ## SARS Coronavirus 2 Positive
        #     oneOutput: "CHEBI:119915"       ## fentanyl

In [341]:
# the following code writes the x-bte-response-maping of ehr_risk_kg.yaml to an output file titled 'x-bte-kgs-operations-response-mapping.txt'
# copy paste the output to the relevant section in the yaml
with open('x-bte-kgs-operations-response-mapping.txt', 'a') as file:
    file.write("  " + "x-bte-response-mapping:" + "\n")
    for sprefix in list(kg_yaml['subject_prefix'].unique()):
        file.write(f"    subject-{sprefix}:" + "\n")
        file.write(f"      {sprefix}: subject.{sprefix}" + "\n")
        file.write(f"      input_name: subject.name" + "\n")
        file.write(f"      output_name: object.name" + "\n")
        file.write(f"      edge-attributes: sources.edge_attributes" + "\n")
        file.write(f"      trapi_sources: source.edge_sources" + "\n")
    for oprefix in list(kg_yaml['object_prefix'].unique()):
        file.write(f"    object-{oprefix}:" + "\n")
        file.write(f"      {oprefix}: object.{oprefix}" + "\n")
        file.write(f"      input_name: object.name" + "\n")
        file.write(f"      output_name: subject.name" + "\n")
        file.write(f"      edge-attributes: sources.edge_attributes" + "\n")
        file.write(f"      trapi_sources: source.edge_sources" + "\n")


In [325]:
# # check to see how many identical triples there are in KG (requested by BTE)
# identical_triples_check = kg.groupby(['subject','object', 'predicate']).size().reset_index().rename(columns={0:'count'}).sort_values(by=['count'], ascending=False)
# identical_triples_check

,subject,object,predicate,count
111072,HP:0001974,NCIT:C167118,biolink:associated_with_decreased_likelihood_of,4
211014,NCIT:C167118,HP:0001974,biolink:associated_with_decreased_likelihood_of,4
117058,HP:0002900,NCIT:C167118,biolink:associated_with_increased_likelihood_of,4
122579,HP:0003124,NCIT:C167118,biolink:associated_with_decreased_likelihood_of,4
117810,HP:0002902,NCIT:C167118,biolink:associated_with_increased_likelihood_of,3
...,...,...,...,...
78133,CHEBI:77431,MONDO:0004721,biolink:associated_with_decreased_likelihood_of,1
78134,CHEBI:77431,MONDO:0004866,biolink:associated_with_decreased_likelihood_of,1
78135,CHEBI:77431,MONDO:0004868,biolink:associated_with_increased_likelihood_of,1
78136,CHEBI:77431,MONDO:0004946,biolink:associated_with_increased_likelihood_of,1


# Make a cleaned version of the parser below

In [32]:
import pandas as pd
import json
import sys, os
import numpy as np

def parse_ehr_risk(data_folder):

    edges_filename = "ehr_risk_edges_data_2022_06_01.csv"
    nodes_filename = "ehr_risk_nodes_data_2022_06_01.csv"

    nodes_filepath = os.path.join(data_folder, nodes_filename)
    edges_filepath = os.path.join(data_folder, edges_filename)
    nodes_data = pd.read_csv(nodes_filepath, sep = ',')
    edges_data = pd.read_csv(edges_filepath, sep = ',')
    
    # the nodes file has duplicate ids; fix in enclave in future
    nodes_data = nodes_data.drop_duplicates(subset='id', keep="first")
    
    # biolink category biolink:ChemicalSubstance has been deprecated. Use biolink:ChemicalEntity instead
    nodes_data["category"].mask(nodes_data["category"] == "biolink:ChemicalSubstance", "biolink:ChemicalEntity" , inplace=True )

    # we originally provided the # of patients with condition --> log + patient count, and # of patients without condition --> log - patient count
    # get the approximate total number of patients in the study and call it "total_sample_size"
    edges_data["num_patients_with_condition"] = 10**(edges_data['log_positive_patient_count']) # convert log pos patient count to an actual # 
    edges_data["num_patients_without_condition"] = 10**(edges_data['log_negative_patient_count']) # convert log neg patient count to an actual #
    edges_data = edges_data.drop(['log_positive_patient_count', 'log_negative_patient_count'], axis=1)
    edges_data["total_sample_size"] = edges_data["num_patients_with_condition"] + edges_data["num_patients_without_condition"]
    edges_data = edges_data.drop(['num_patients_with_condition', 'num_patients_without_condition'], axis=1)
    edges_data["total_sample_size"] = np.random.poisson(edges_data["total_sample_size"]) # add poisson noise injection 

#     # create confidence interval column by concatenating 'lower_confidence_bound'and 'upper_confidence_bound', then dropping those columns
#     edges_data['log_odds_ratio_95_confidence_interval'] = edges_data.apply(lambda row: [row['lower_confidence_bound'], row['upper_confidence_bound']], axis=1)
#     edges_data = edges_data.drop(['lower_confidence_bound', 'upper_confidence_bound'], axis=1)
    
    #  ----- RE-CONSTRUCT KG FROM NODES AND EDGES FILES ------  #
    # merge the subject names, categories and ids from the nodes csv/table to the edges table
    kg = pd.merge(edges_data, nodes_data[['id', 'name', 'category']], left_on='subject', right_on = 'id', how="inner")
    kg.rename(columns = {'category_x':'predicate_category',
                         'category_y': 'subject_category',
                         'id': 'subject_id',
                         'name': 'subject_name'}, inplace = True)
    # merge the object names, categories and ids from the nodes csv/table to the edges table
    kg = pd.merge(kg, nodes_data[['id', 'name', 'category']], left_on='object', right_on = 'id', how="inner")
    kg.rename(columns = {'id':'object_id',
                     'category': 'object_category',
                     'name': 'object_name'}, inplace = True)
    #  ----- ------------------------------------------ ------  #
    
    # ensure there are no duplicates
    kg = kg.drop_duplicates(['subject', 'object', 'auc_roc', 'p_value', 'feature_coefficient'], keep='first')
    
    # some of the subjects/objects contain the string literal "NONE" (specific culprit is COVID Negative or something) Should look into this in future 
    kg = kg[~kg["subject"].str.contains("NONE")==True]  # subject and object are all CURIEs, not names
    kg = kg[~kg["object"].str.contains("NONE")==True]
    kg = kg[~kg["subject"].str.contains("none")==True]
    kg = kg[~kg["object"].str.contains("none")==True]
    kg = kg[~kg["subject"].str.contains("None")==True]
    kg = kg[~kg["object"].str.contains("None")==True]
    
    id_list = [] # use this to check if your document IDs are unique. Collect them and see if they're all unique
    
    # iterate through each row in KG to yield json formatted triple
    for index, row in kg[:40].iterrows(): # comment for testing  
        id_dict = {} # this is the outter dict that holds inner dicts: subject_dict, association_dict, object_dict, and source_dict
        subject_dict = {} # inner dict
        association_dict = {} # inner dict
        object_dict = {} # inner dict
        source_dict = {} # inner dict (provides provenance as per TRAPI 1.4 standards)

        # id generated by concatenating the following: subject_id CURIE, object_id CURIE, AUCROC (removing decimal point) and p-value (removing decimal point), feature coeffcient (removing decimal point), and total sample size
        doc_id = "{}_{}_{}_{}_{}_{}".format(row["subject"],
                                       row["object"],
                                       str(row['auc_roc']),
                                       str(row['p_value']).replace('.', ''),
                                       str(row['feature_coefficient']).replace('.', ''),
                                       str(row["total_sample_size"]))

        id_list.append(doc_id)
        id_dict["_id"] = doc_id
        subject_dict["{}".format(row["subject"].split(':')[0])] = "{}".format(row["subject"].split(':')[1]) # create the subject dict from the rows of the df 
        subject_dict["id"] = row["subject"]
        subject_dict["name"] = row["subject_name"]
        subject_dict["type"] = row["subject_category"]

        association_dict["predicate"] = "{}".format(row["predicate"].split(':')[1]) # create the association dict from the rows of the df. Edge attributes need extra work. The predicate is separated out into qualified predicate by X-BTE annotation, so we don't have to worry about qualifiers here
        association_dict["edge_attributes"] = []

        source_dict["edge_sources"] = []

        association_dict["edge_attributes"].append(
            {
                "attribute_type_id":"biolink:has_supporting_study_result",
                "value":"We train a large collection of multivariable, binary logistic regression models on EHR data for each specific condition/disease/outcome. Features include labs, medications, and phenotypes. Directed edges point from risk factors to specific outcomes (diseases, phenotype, or medication exposure).",
                "attributes": [
                    {
                        "attribute_type_id": "biolink:supporting_study_method_type",
                        "value": "STATO:0000149",
                        "description": "Binomial logistic regression for analysis of dichotomous dependent variable (in this case, for having this particular condition/disease/outcome or not)"
                    },
                    {
                        "attribute_type_id":"biolink:update_date",
                        "value":row["provided_date"]
                    },
                    {
                        "attribute_type_id": "biolink:p_value",
                        "value": row["p_value"],
                        "description": "The p-value represents the probability of observing the estimated coefficient (or more extreme value) under the assumption of the null hypothesis (which assumes that there is no relationship between the independent variable and outcome variable). The p-value associated with each coefficient helps determine whether the relationship between the independent variable and the outcome is statistically significant. A low p-value suggests that the observed relationship between the independent variable and the outcome is unlikely to occur by chance alone, providing evidence against the null hypothesis."
                    },
                    {
                        "attribute_type_id": "STATO:0000209",
                        "value": row["auc_roc"],
                        "description": "The AUROC provides a way to evaluate the model's ability to discriminate between the two classes (the presenece of absence of condition/disease/outcome). Values range between 0-1; the higher the AUROC, the better the model's ability to discriminate between clasess."
                    },
                    {
                        "attribute_type_id": "biolink:log_odds_ratio",
                        "value": row['feature_coefficient'],
                        "description": "The logarithm of the odds ratio (log odds ratio), or the ratio of the odds of event Y occurring in an exposed group versus the odds of event Y occurring in a non-exposed group."
                    },
#                     {
#                         "attribute_type_id": "biolink:log_odds_ratio_95_confidence_interval",
#                         "value": row['log_odds_ratio_95_confidence_interval'],
#                         "description": "log_odds_ratio_95_confidence_interval"
#                     },
                    {
                        "attribute_type_id": "biolink:supporting_study_cohort",
                        "value": "age < 18 excluded"
                    },
                    {
                        "attribute_type_id": "biolink:supporting_study_date_range",
                        "value": "2020-2022 (prediction)"
                    },
                    {
                        "attribute_type_id": "biolink:total_sample_size",
                        "value": row["total_sample_size"],
                        "description": "The total number of patients or participants within a sample population."
                    }
                ]
            }
        )
        association_dict["edge_attributes"].append(
            {
                "attribute_type_id":"biolink:primary_knowledge_source",
                "value":"infores:biothings-multiomics-ehr-risk",
                "value_type_id": "biolink:InformationResource",
                "value_url":  "http://smart-api.info/registry?q=d86a24f6027ffe778f84ba10a7a1861a",
                "description": "The EHR Risk KP is created and maintained by the Multiomics Provider team from the Institute for Systems Biology in Seattle, WA. Through a partnership with Providence/Swedish Health Services and Institute for Systems Biology, we analyze over 26 million EHRs. We use these records to train a large collection of interpretable machine learning models which are integrated into a single large Knowledge Graph, with directed edges pointing from risk factors to specific outcomes (diseases, phenotype, or medication exposure).",
            }
        )
        association_dict["edge_attributes"].append(
            {
                "attribute_type_id":"biolink:supporting_data_source",
                "value":"infores:providence-st-joseph-ehr",
                "value_type_id": "biolink:InformationResource",
                "value_url":  "https://github.com/NCATSTranslator/Translator-All/wiki/EHR-Risk-KP",
                "description": "A partnership with Providence/Swedish Health Services and Institute for Systems Biology allows analysis of 26 million EHRs from patients in seven states in the US, including Alaska, California, Montana, Oregon, Washington, Texas, and New Mexico. Please email data-access@isbscience.org for more information.",
            }
        )

        object_dict["{}".format(row["object"].split(':')[0])] = "{}".format(row["object"].split(':')[1]) # create the object dict from the rows of the df 
        object_dict["id"] = row["object"]
        object_dict["name"] = row["object_name"]
        object_dict["type"] = row["object_category"]

        source_dict["edge_sources"].append(
            {
                "resource_id": "infores:biothings-multiomics-ehr-risk",
                "resource_role": "primary_knowledge_source",
                "upstream_resource_ids": "infores:providence-st-joseph-ehr"
            }
        )

        source_dict["edge_sources"].append(
            {
                "resource_id": "infores:providence-st-joseph-ehr",
                "resource_role": "supporting_data_source"
            }
        )

        id_dict["subject"] = subject_dict # put the subject, association, object, and source dicts into the outer dict called id_dict
        id_dict["association"] = association_dict
        id_dict["object"] = object_dict
        id_dict["source"] = source_dict
            
        # throw error for any rows that are missing any relevant values, such as subject name, subject id/CURIE, subject category, p-value, etc...
        try:
            assert not {x for x in {row["total_sample_size"],
                                    row["subject"],
                                    row["subject_name"],
                                    row["subject_category"],
                                    row["object"],
                                    row["object_name"],
                                    row["object_category"],
                                    row["p_value"],
                                    row["auc_roc"],
                                    row['feature_coefficient']} if x in {None,
                                                                         "NONE",
                                                                         "None",
                                                                         "none",
                                                                         "NA"}}, "Error: All values including subject and object IDs, categories, names, p-value, AUC-ROC, and feature coefficient must be non-null and not contain string literal None or NONE"
            print(json.dumps(id_dict, indent=2)) # uncomment for testing
#             print(index) # uncomment for testing
#             yield id_dict # comment for testing
        except AssertionError as msg:
            print(msg)

    if len(id_list) != len(set(id_list)):
        print("You do not have unique document IDs for each edge in your KG. Either you have duplicate rows/edges, or you simply didn't make a unique identifer (Document ID) for each one.\n\n\n")
    else:
        print("Document IDs appear to be unique\n\n\n")
        
        
    

{
  "_id": "HP:0008629_HP:0000360_0.8401321539277617_00_8796399245685702_10102731",
  "subject": {
    "HP": "0008629",
    "id": "HP:0008629",
    "name": "Pulsatile tinnitus",
    "type": "biolink:PhenotypicFeature"
  },
  "association": {
    "predicate": "associated_with_increased_likelihood_of",
    "edge_attributes": [
      {
        "attribute_type_id": "biolink:has_supporting_study_result",
        "value": "We train a large collection of multivariable, binary logistic regression models on EHR data for each specific condition/disease/outcome. Features include labs, medications, and phenotypes. Directed edges point from risk factors to specific outcomes (diseases, phenotype, or medication exposure).",
        "attributes": [
          {
            "attribute_type_id": "biolink:supporting_study_method_type",
            "value": "STATO:0000149",
            "description": "Binomial logistic regression for analysis of dichotomous dependent variable (in this case, for having this

In [ ]:
import pandas as pd
import json
import sys, os
import numpy as np

def parse_ehr_risk(data_folder):

    edges_filename = "ehr_risk_edges_data_2022_06_01.csv"
    nodes_filename = "ehr_risk_nodes_data_2022_06_01.csv"

    nodes_filepath = os.path.join(data_folder, nodes_filename)
    edges_filepath = os.path.join(data_folder, edges_filename)
    nodes_data = pd.read_csv(nodes_filepath, sep = ',')
    edges_data = pd.read_csv(edges_filepath, sep = ',')
    
    # the nodes file has duplicate ids; fix in enclave in future
    nodes_data = nodes_data.drop_duplicates(subset='id', keep="first")
    
    # biolink category biolink:ChemicalSubstance has been deprecated. Use biolink:ChemicalEntity instead
    nodes_data["category"].mask(nodes_data["category"] == "biolink:ChemicalSubstance", "biolink:ChemicalEntity" , inplace=True )

    # we originally provided the # of patients with condition --> log + patient count, and # of patients without condition --> log - patient count
    # get the approximate total number of patients in the study and call it "total_sample_size"
    edges_data["num_patients_with_condition"] = 10**(edges_data['log_positive_patient_count']) # convert log pos patient count to an actual # 
    edges_data["num_patients_without_condition"] = 10**(edges_data['log_negative_patient_count']) # convert log neg patient count to an actual #
    edges_data = edges_data.drop(['log_positive_patient_count', 'log_negative_patient_count'], axis=1)
    edges_data["total_sample_size"] = edges_data["num_patients_with_condition"] + edges_data["num_patients_without_condition"]
    edges_data = edges_data.drop(['num_patients_with_condition', 'num_patients_without_condition'], axis=1)
    edges_data["total_sample_size"] = np.random.poisson(edges_data["total_sample_size"]) # add poisson noise injection 

#     # create confidence interval column by concatenating 'lower_confidence_bound'and 'upper_confidence_bound', then dropping those columns
#     edges_data['log_odds_ratio_95_confidence_interval'] = edges_data.apply(lambda row: [row['lower_confidence_bound'], row['upper_confidence_bound']], axis=1)
#     edges_data = edges_data.drop(['lower_confidence_bound', 'upper_confidence_bound'], axis=1)
    
    #  ----- RE-CONSTRUCT KG FROM NODES AND EDGES FILES ------  #
    # merge the subject names, categories and ids from the nodes csv/table to the edges table
    kg = pd.merge(edges_data, nodes_data[['id', 'name', 'category']], left_on='subject', right_on = 'id', how="inner")
    kg.rename(columns = {'category_x':'predicate_category',
                         'category_y': 'subject_category',
                         'id': 'subject_id',
                         'name': 'subject_name'}, inplace = True)
    # merge the object names, categories and ids from the nodes csv/table to the edges table
    kg = pd.merge(kg, nodes_data[['id', 'name', 'category']], left_on='object', right_on = 'id', how="inner")
    kg.rename(columns = {'id':'object_id',
                     'category': 'object_category',
                     'name': 'object_name'}, inplace = True)
    #  ----- ------------------------------------------ ------  #
    
    # ensure there are no duplicates
    kg = kg.drop_duplicates(['subject', 'object', 'auc_roc', 'p_value', 'feature_coefficient'], keep='first')
    
    # some of the subjects/objects contain the string literal "NONE" (specific culprit is COVID Negative or something) Should look into this in future 
    kg = kg[~kg["subject"].str.contains("NONE")==True]  # subject and object are all CURIEs, not names
    kg = kg[~kg["object"].str.contains("NONE")==True]
    kg = kg[~kg["subject"].str.contains("none")==True]
    kg = kg[~kg["object"].str.contains("none")==True]
    kg = kg[~kg["subject"].str.contains("None")==True]
    kg = kg[~kg["object"].str.contains("None")==True]
    
    id_list = [] # use this to check if your document IDs are unique. Collect them and see if they're all unique
    
    # iterate through each row in KG to yield json formatted triple
    for index, row in kg[:40].iterrows(): # comment for testing  
        id_dict = {} # this is the outter dict that holds inner dicts: subject_dict, association_dict, object_dict, and source_dict
        subject_dict = {} # inner dict
        association_dict = {} # inner dict
        object_dict = {} # inner dict
        source_dict = {} # inner dict (provides provenance as per TRAPI 1.4 standards)

        # id generated by concatenating the following: subject_id CURIE, object_id CURIE, AUCROC (removing decimal point) and p-value (removing decimal point), feature coeffcient (removing decimal point), and total sample size
        doc_id = "{}_{}_{}_{}_{}_{}".format(row["subject"],
                                       row["object"],
                                       str(row['auc_roc']),
                                       str(row['p_value']).replace('.', ''),
                                       str(row['feature_coefficient']).replace('.', ''),
                                       str(row["total_sample_size"]))

        id_list.append(doc_id)
        id_dict["_id"] = doc_id
        subject_dict["{}".format(row["subject"].split(':')[0])] = "{}".format(row["subject"].split(':')[1]) # create the subject dict from the rows of the df 
        subject_dict["id"] = row["subject"]
        subject_dict["name"] = row["subject_name"]
        subject_dict["type"] = row["subject_category"]

        association_dict["predicate"] = "{}".format(row["predicate"].split(':')[1]) # create the association dict from the rows of the df. Edge attributes need extra work. The predicate is separated out into qualified predicate by X-BTE annotation, so we don't have to worry about qualifiers here
        association_dict["edge_attributes"] = []

        source_dict["edge_sources"] = []

        association_dict["edge_attributes"].append(
            {
                "attribute_type_id":"biolink:has_supporting_study_result",
                "value":"We train a large collection of multivariable, binary logistic regression models on EHR data for each specific condition/disease/outcome. Features include labs, medications, and phenotypes. Directed edges point from risk factors to specific outcomes (diseases, phenotype, or medication exposure).",
                "attributes": [
                    {
                        "attribute_type_id": "biolink:supporting_study_method_type",
                        "value": "STATO:0000149",
                        "description": "Binomial logistic regression for analysis of dichotomous dependent variable (in this case, for having this particular condition/disease/outcome or not)"
                    },
                    {
                        "attribute_type_id":"biolink:update_date",
                        "value":row["provided_date"]
                    },
                    {
                        "attribute_type_id": "biolink:p_value",
                        "value": row["p_value"],
                        "description": "The p-value represents the probability of observing the estimated coefficient (or more extreme value) under the assumption of the null hypothesis (which assumes that there is no relationship between the independent variable and outcome variable). The p-value associated with each coefficient helps determine whether the relationship between the independent variable and the outcome is statistically significant. A low p-value suggests that the observed relationship between the independent variable and the outcome is unlikely to occur by chance alone, providing evidence against the null hypothesis."
                    },
                    {
                        "attribute_type_id": "STATO:0000209",
                        "value": row["auc_roc"],
                        "description": "The AUROC provides a way to evaluate the model's ability to discriminate between the two classes (the presenece of absence of condition/disease/outcome). Values range between 0-1; the higher the AUROC, the better the model's ability to discriminate between clasess."
                    },
                    {
                        "attribute_type_id": "biolink:log_odds_ratio",
                        "value": row['feature_coefficient'],
                        "description": "The logarithm of the odds ratio (log odds ratio), or the ratio of the odds of event Y occurring in an exposed group versus the odds of event Y occurring in a non-exposed group."
                    },
#                     {
#                         "attribute_type_id": "biolink:log_odds_ratio_95_confidence_interval",
#                         "value": row['log_odds_ratio_95_confidence_interval'],
#                         "description": "log_odds_ratio_95_confidence_interval"
#                     },
                    {
                        "attribute_type_id": "biolink:supporting_study_cohort",
                        "value": "age < 18 excluded"
                    },
                    {
                        "attribute_type_id": "biolink:supporting_study_date_range",
                        "value": "2020-2022 (prediction)"
                    },
                    {
                        "attribute_type_id": "biolink:total_sample_size",
                        "value": row["total_sample_size"],
                        "description": "The total number of patients or participants within a sample population."
                    }
                ]
            }
        )
        association_dict["edge_attributes"].append(
            {
                "attribute_type_id":"biolink:primary_knowledge_source",
                "value":"infores:biothings-multiomics-ehr-risk",
                "value_type_id": "biolink:InformationResource",
                "value_url":  "http://smart-api.info/registry?q=d86a24f6027ffe778f84ba10a7a1861a",
                "description": "The EHR Risk KP is created and maintained by the Multiomics Provider team from the Institute for Systems Biology in Seattle, WA. Through a partnership with Providence/Swedish Health Services and Institute for Systems Biology, we analyze over 26 million EHRs. We use these records to train a large collection of interpretable machine learning models which are integrated into a single large Knowledge Graph, with directed edges pointing from risk factors to specific outcomes (diseases, phenotype, or medication exposure).",
            }
        )
        association_dict["edge_attributes"].append(
            {
                "attribute_type_id":"biolink:supporting_data_source",
                "value":"infores:providence-st-joseph-ehr",
                "value_type_id": "biolink:InformationResource",
                "value_url":  "https://github.com/NCATSTranslator/Translator-All/wiki/EHR-Risk-KP",
                "description": "A partnership with Providence/Swedish Health Services and Institute for Systems Biology allows analysis of 26 million EHRs from patients in seven states in the US, including Alaska, California, Montana, Oregon, Washington, Texas, and New Mexico. Please email data-access@isbscience.org for more information.",
            }
        )

        object_dict["{}".format(row["object"].split(':')[0])] = "{}".format(row["object"].split(':')[1]) # create the object dict from the rows of the df 
        object_dict["id"] = row["object"]
        object_dict["name"] = row["object_name"]
        object_dict["type"] = row["object_category"]

        source_dict["edge_sources"].append(
            {
                "resource_id": "infores:biothings-multiomics-ehr-risk",
                "resource_role": "primary_knowledge_source",
                "upstream_resource_ids": "infores:providence-st-joseph-ehr"
            }
        )

        source_dict["edge_sources"].append(
            {
                "resource_id": "infores:providence-st-joseph-ehr",
                "resource_role": "supporting_data_source"
            }
        )

        id_dict["subject"] = subject_dict # put the subject, association, object, and source dicts into the outer dict called id_dict
        id_dict["association"] = association_dict
        id_dict["object"] = object_dict
        id_dict["source"] = source_dict
            
        # throw error for any rows that are missing any relevant values, such as subject name, subject id/CURIE, subject category, p-value, etc...

        try:
            assert not {x for x in {row["total_sample_size"],
                                    row["subject"],
                                    row["subject_name"],
                                    row["subject_category"],
                                    row["object"],
                                    row["object_name"],
                                    row["object_category"],
                                    row["p_value"],
                                    row["auc_roc"],
                                    row['feature_coefficient']} if x in {None,
                                                                         "NONE",
                                                                         "None",
                                                                         "none",
                                                                         "NA"}}, "Error: All values including subject and object IDs, categories, names, p-value, AUC-ROC, and feature coefficient must be non-null and not contain string literal None or NONE"
            print(json.dumps(id_dict, indent=2)) # uncomment for testing
#             print(index) # uncomment for testing
#             yield id_dict # comment for testing
        except AssertionError as msg:
            print(msg)

    if len(id_list) != len(set(id_list)):
        print("You do not have unique document IDs for each edge in your KG. Either you have duplicate rows/edges, or you simply didn't make a unique identifer (Document ID) for each one.\n\n\n")
    else:
        print("Document IDs appear to be unique\n\n\n")




    

In [151]:
def main():
    data_folder = "../../data" # uncomment for testing
    parse_ehr_risk(data_folder) 

    
if __name__ == "__main__":
    main()

# def f(): return list(parse_ehr_risk(data_folder))

# print(f())

Document IDs appear to be unique

[{'_id': 'HP:0008629_HP:0000360_0.8401321539277617_00_8796399245685702_10096539', 'subject': {'HP': '0008629', 'id': 'HP:0008629', 'name': 'Pulsatile tinnitus', 'type': 'biolink:PhenotypicFeature'}, 'association': {'predicate': 'associated_with_increased_likelihood_of', 'edge_attributes': [{'attribute_type_id': 'biolink:has_supporting_study_result', 'value': 'We train a large collection of multivariable, binary logistic regression models on EHR data for each specific condition/disease/outcome. Features include labs, medications, and phenotypes. Directed edges point from risk factors to specific outcomes (diseases, phenotype, or medication exposure).', 'attributes': [{'attribute_type_id': 'biolink:supporting_study_method_type', 'value': 'STATO:0000149', 'description': 'Binomial logistic regression for analysis of dichotomous dependent variable (in this case, for having this particular condition/disease/outcome or not)'}, {'attribute_type_id': 'biolink:u

In [ ]:
### from UI, we found duplicate predicate in June 6 2020 edges files

In [ ]:
edges_data.loc[(edges_data['subject'] == 'MONDO:0011849') & (edges_data['object'] == 'MONDO:0005083')]

In [ ]:
edges_data.loc[(edges_data['subject'] == 'MONDO:0005083') & (edges_data['object'] == 'MONDO:0011849')]

In [ ]:
https://github.com/uhbrar/ReasonerAPI/blob/update_guide/MigrationAndImplementationGuide1-4.md

In [ ]:
variable names to use for Clinical Data Committee:
log_odds_ratio, total_sample_size, log_odds_ratio_95_confidence_interval

In [ ]:
# example 1.4 output from molepro
{
  	"attribute_type_id": "biolink:aggregator_knowledge_source",	
  	"value": "infores:molepro",      
  	"value_type_id": "biolink:InformationResource",    
  	"value_url":  "https://translator.broadinstitute.org/molepro/trapi/v1.0",
  	"description": "The Molecular Data Provider KP from NCATS Translator",
  	"attribute_source": "infores:molepro"
       },
	{
  	"attribute_type_id": "biolink:aggregator_knowledge_source",
  	"value":  "infores:chembl",
  	"value_type_id": "biolink:InformationResource",    
  	"value_url":  "https://www.ebi.ac.uk/chembl",
  	"description": "ChEMBL is a manually curated database of bioactive molecules...",
  	"attribute_source": "infores:molepro"
	},
	{
  	"attribute_type_id": "biolink:primary_knowledge_source",
  	"value":  "infores:clinical-trials-gov", 
  	"value_type_id": "biolink:InformationResource",    
  	"value_url":  "https://www.clinicaltrials.gov",
  	"description": "ClinicalTrials.gov is...",
  	"attribute_source": "infores:chembl"


In [ ]:
"edges": [
  {
  "id": "Association002",   
  "category": "biolink:FeatureVariableAssociation",
  "subject": "ncit:C29886" # Airborne Particulate Matter (PM2.5),   	 
  "predicate": "biolink:correlates_with",
  "object": "tvfo:xxxxx" # t.b.d. term for 'ED Visits for Asthma',
  "subject_modifier": "biolink:Exposure",
  "attributes":  [
	{
  	"attribute_type_id": "biolink:primary_knowledge_source",  
  	"value":  "infores:icees-asthma",  
  	"value_type_id": "biolink:InformationResource",	 
  	"value_url":  "https://icees.renci.org:16339",
  	"description": "The ICEES Provider ...",
  	"attribute_source": "infores:icees-asthma"
	},
	{
  	"attribute_type_id": "biolink:supporting_data_source",  
  	"value":  "infores:us-epa-airborne-pollutant-exposures-data",
  	"value_type_id": "biolink:InformationResource",	 
  	"description": "US Environmental Protection Agency Airborne Pollutant Exposure Data",
  	"attribute_source": "infores:icees-asthma"
	},
	{
  	"attribute_type_id": "biolink:supporting_data_source",  
  	"value":  "infores:unc-cdw-health",
  	"value_type_id": "biolink:InformationResource",	 
  	"description": "UNC Carolina Data Warehouse for Health Patient EHR Data",
  	"attribute_source": "infores:icees-asthma"
	}
     ]
  }
]

In [ ]:
edges_sig["num_patients_with_condition"] = 10**(edges_sig['log_positive_patient_count'])
edges_sig["num_patients_without_condition"] = 10**(edges_sig['log_negative_patient_count'])
edges_sig